# Clases en tiempos modernos

Aunque definir clases es uno de los paradigmas más viejos en la programación, no ha perdido vigencia y se ido modernizando poco a poco a través de las versiones más actuales de Python. Uno de los aspectos más destabales es el salto en delcaración de clases, ya que utilizar el método `__init__()` muchas veces resulta confuso, por lo que en versiones modernas se implmenta el uso de `dataclasses`, permitiendo simplificar la construcción de una clase

```python
from dataclasses import dataclass

@dataclass
class NombreClase:
    
    atributo: anotación
    atributo: anotación = valor
    
    def método (self, argumentos):
        sentencias
```

Nótese que ha desaparecido el uso del méotod mágico `__init__()`, y en su lugar la declaración de atributos queda de una forma directa como si fueran atributos de calse, con la diferencia de que admite valores de entrada, por ejemplo:

In [1]:
# Caso anterior
class Bicicleta:

	def __init__(self, color: str, rodada: float, tipo: str):
		self.color = color
		self.rodada = rodada
		self.tipo = tipo

In [3]:
from dataclasses import dataclass

@dataclass
class Bicicleta:
    
    color: str
    rodada:float
    tipo: str

El objeto `dataclass` es un método de la paquetería `dataclasses`, y es llamado como decorador de nuestra clase (`Bicicleta` en el ejemplo anterior), utlizando la estructura `@dataclass`

### Decoradores

Supongamos que tenemos un conjunto de funciones, que realizan ciertas sentencias particulares, por ejemplo, operaciones matemáticas entre dos números:

In [8]:
def suma(a:int, b:int):
    return a+b

def resta(a:int, b:int):
    return a-b

# ··· Son muchas

Ahora supongamos que requerimos que siempre, antes de realizar la operación y regresar el resultado, necesitamos que se imprima un saludo y al final una despedida.

In [ ]:
def suma(a:int, b:int):
    print("Hola, mundo!")
    c = a+b
    print("Adios, mundo!")
    return c

def resta(a:int, b:int):
    print("Hola, mundo!")
    c = a-b
    print("Adios, mundo!")
    return c

Llamamos a un decorador. Un decorador es una función que se aplica sobre otra función.

In [15]:
def decorador_impresion(funcion):
    def wrapper(*args):
        print("Hola, mundo!")
        result = funcion(*args)
        print(result)
        print("Adios, mundo!")
        return result
        
    return wrapper

In [10]:
def suma(a:int, b:int):
    return a+b

def resta(a:int, b:int):
    return a-b

In [11]:
suma(1,2)

3

In [12]:
resta(1,2)

-1

In [16]:
@decorador_impresion
def suma(a:int, b:int):
    return a+b

@decorador_impresion
def resta(a:int, b:int):
    return a-b

In [18]:
resultado = suma(1,2)

Hola, mundo!
3
Adios, mundo!


In [19]:
resultado = resta(1,2)

Hola, mundo!
-1
Adios, mundo!


In [26]:
# Ejemplo de clases con decoradores
from dataclasses import dataclass

@dataclass
class OperacionesBasicas:
    
    a:int
    b:int
    
    @decorador_impresion
    def suma(self):
        return self.a + self.b
    
    @decorador_impresion
    def resta(self):
        return self.a - self.b
    
    @decorador_impresion
    def producto(self):
        return self.a * self.b
    
    @decorador_impresion
    def division(self):
        return self.a / self.b
    
    @decorador_impresion
    def potencia(self):
        return self.a ** self.b

In [37]:
op = OperacionesBasicas(3,2)

In [38]:
op.suma()

Hola, mundo!
5
Adios, mundo!


5

In [39]:
op.potencia()

Hola, mundo!
9
Adios, mundo!


9

# Validaciones

Hemos visto como al utilizar `@dataclass` simplifica la construcción de clases sin afectar su llamado durante la creación de objetos. Al usar anotaciones en una clase declara con `@dataclass`, nos permite saber cómo debemos llamar la clase, sin embargo ¿son suficientes las anotaciones para controlar erroes?. La respuesta es **no**, si bien ayudan durante el desarrollo, existen casos, aún dentro del desarrollo, en que no se consulta la docuemtanció nde una clase (mala práctica común), por lo que agregar una capa de seguirdad en la integridad de los datos ayudaría bastante

In [40]:
# Problemática
op = OperacionesBasicas("1", "2") # Mando a y b como str, aunque la anotación es int, no regresa ningún error

In [42]:
# Eso es un problema porque al llamar a los métodos o simplemente usar el objeto `op`, podemos pensar que si va a
# funcionar bien pero no es así siempre
op.suma() # funciona, no saca error, pero el resultado no es correcto

Hola, mundo!
12
Adios, mundo!


'12'

In [43]:
op.potencia() # O simplemente saca error

Hola, mundo!


TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'str'

Lo ideal sería cachar el error desde que se inicializa el objeto.

Para lo anterior existe una paquetría de terceros que se llama **Pydantic**, una librería de validación que aporta bastante en la validación de datos. Para instalar pydantic usamos:

```bash
$ pip install pydantic

$ poetry add pydantic
```

In [45]:
# Se puede correr comandos de bash desde el notebook
!pip install pydantic

Defaulting to user installation because normal site-packages is not writeable


In [54]:
from pydantic.dataclasses import dataclass

@dataclass
class Calculadora:
    
    a:int
    b:int
    
    @decorador_impresion
    def suma(self):
        return self.a + self.b
    
    @decorador_impresion
    def resta(self):
        return self.a - self.b
    
    @decorador_impresion
    def producto(self):
        return self.a * self.b
    
    @decorador_impresion
    def division(self):
        return self.a / self.b
    
    @decorador_impresion
    def potencia(self):
        return self.a ** self.b

In [55]:
op = Calculadora(1,2)

In [57]:
op.suma()

Hola, mundo!
3
Adios, mundo!


3

In [56]:
op = Calculadora("1","2")

In [58]:
op.suma()

Hola, mundo!
3
Adios, mundo!


3

En este caso, instanciamos el objeto utilizando tanto `int`, que es la anotación que se pide en la clase directamente, y también usando `str`. Pero en este caso, el objeto que se inicializó con los strings, sí regresó el resultado correcto, en lugar de concatenarlos como se hizo anteriormente. 

Esto sucede porque Pydantic, al recibir cualquier objeto donde el tipado es `int` lo primero que va a hacer es ver si lo puede convertir a esta anotación

In [60]:
type(int("234"))

int

In [61]:
op = Calculadora([1,2],(3,4))

ValidationError: 2 validation errors for Calculadora
a
  value is not a valid integer (type=type_error.integer)
b
  value is not a valid integer (type=type_error.integer)

En el caso anterior, los atributos de entrada no se pudieron convertir a `int` por lo que saltó un `ValidationError`, que nos indica que los atributos de entra `a` y `b` no son objetos `int` válidos.

## Validación directa de atributos

```python
from pydantic.dataclasses import dataclass
from pydantic import validator

@dataclass
class Nombre_de_clase:
	atributos: tipo

	@validator("atributo")
	def nombre_validador(parámetro):
		... validaciones
		return parámetro
```

Supongamos que en mi clase Calculadora, necesito asegurarme que tanto `a` como `b` sean mayor a 0

In [92]:
from pydantic.dataclasses import dataclass
from pydantic import validator

@dataclass
class Calculadora:
    
    a:int
    b:int
    
    @decorador_impresion
    def suma(self):
        return self.a + self.b
    
    @decorador_impresion
    def resta(self):
        return self.a - self.b
    
    @decorador_impresion
    def producto(self):
        return self.a * self.b
    
    @decorador_impresion
    def division(self):
        return self.a / self.b
    
    @decorador_impresion
    def potencia(self):
        return self.a ** self.b
    
    @validator("a")
    def __validador_a(a:int):
        if a > 0:
            return a
        else:
            raise ValueError("a no es mayor a 0")
            
    @validator("b")
    def __validador_b(b:int):
        if b < -15:
            return b
        else:
            raise ValueError("b no es mayor a 0")

In [88]:
calc = Calculadora(1,-27)

In [89]:
calc = Calculadora(-2,2)

ValidationError: 2 validation errors for Calculadora
a
  a no es mayor a 0 (type=value_error)
b
  b no es mayor a 0 (type=value_error)

In [90]:
calc = Calculadora(-1,-20)

ValidationError: 1 validation error for Calculadora
a
  a no es mayor a 0 (type=value_error)

In [91]:
calc = Calculadora(1,-2)

ValidationError: 1 validation error for Calculadora
b
  b no es mayor a 0 (type=value_error)

### Incialización con Pydantic

Usando Pydantic ya nos quitamos la molestia de andar definiendo nuestros atributos en el método mágico `__init__()`, pero es probable que aún así necesitemos que en la inicialización del objeto se realicen algunas sentencias. Para ello usamos el método mágico `__post_init_post_parse__()`

In [101]:
from pydantic.dataclasses import dataclass
from pydantic import validator, root_validator, validate_arguments

# validator: Valida un atributo a la vez de una clase
# root_validator: Valida todos los atributos a la vez de una clase
# validate_arguments: Valida todos los argumentos (posicionales o pre-definidos) de cualquier función o método

@dataclass
class Calculadora:
    
    a:int
    b:int = -100
    
    def __post_init_post_parse__(self):
        # Sentencias de inicialización
        print(f"Se inicializó el objeto Calculadora con atributos ({self.a},{self.b})")
    
    @decorador_impresion
    def suma(self):
        return self.a + self.b
    
    @decorador_impresion
    def resta(self):
        return self.a - self.b
    
    @decorador_impresion
    def producto(self):
        return self.a * self.b
    
    @decorador_impresion
    def division(self):
        return self.a / self.b
    
    @decorador_impresion
    def potencia(self):
        return self.a ** self.b
    
    @validator("a")
    def __validador_a(a:int):
        if a > 0:
            return a
        else:
            raise ValueError("a no es mayor a 0")
            
    @validator("b")
    def __validador_b(b:int):
        if b < -15:
            return b
        else:
            raise ValueError("b no es mayor a 0")

In [102]:
calc = Calculadora(30, -24)

Se inicializó el objeto Calculadora con atributos (30,-24)
